In [144]:
import pandas as pd
import numpy as np

In [145]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160213.txt')

In [146]:
data.columns

Index([u'C/A', u'UNIT', u'SCP', u'STATION', u'LINENAME', u'DIVISION', u'DATE',
       u'TIME', u'DESC', u'ENTRIES',
       u'EXITS                                                               '],
      dtype='object')

In [147]:
data.rename(columns = {'EXITS                                                               ':'EXITS','DATE':'DATE'}
            ,inplace = True)
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,02/06/2016,03:00:00,REGULAR,5530860,1867106
1,A002,R051,02-00-00,59 ST,NQR456,BMT,02/06/2016,07:00:00,REGULAR,5530874,1867120
2,A002,R051,02-00-00,59 ST,NQR456,BMT,02/06/2016,11:00:00,REGULAR,5530973,1867216
3,A002,R051,02-00-00,59 ST,NQR456,BMT,02/06/2016,15:00:00,REGULAR,5531226,1867280
4,A002,R051,02-00-00,59 ST,NQR456,BMT,02/06/2016,19:00:00,REGULAR,5531634,1867368


In [148]:
# create an identifier for each turnstile at a station
data['TURNSTILE'] = data['UNIT'] + data['SCP']

In [149]:
np.unique(data.DATE)

array(['02/06/2016', '02/07/2016', '02/08/2016', '02/09/2016',
       '02/10/2016', '02/11/2016', '02/12/2016'], dtype=object)

### Stations of interest:
- Bedford Ave
- Lorimer St
- Metropolitan Ave
- Nassau Ave
- Marcy Av

In [150]:
# create a new dataframe with only bedford data
bedford = data[data['STATION']=='BEDFORD AV']
bedford.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TURNSTILE
30297,H009,R235,00-00-00,BEDFORD AV,L,BMT,02/06/2016,03:00:00,REGULAR,2013752,797893,R23500-00-00
30298,H009,R235,00-00-00,BEDFORD AV,L,BMT,02/06/2016,07:00:00,REGULAR,2013825,797913,R23500-00-00
30299,H009,R235,00-00-00,BEDFORD AV,L,BMT,02/06/2016,11:00:00,REGULAR,2014117,797963,R23500-00-00
30300,H009,R235,00-00-00,BEDFORD AV,L,BMT,02/06/2016,15:00:00,REGULAR,2014836,798241,R23500-00-00
30301,H009,R235,00-00-00,BEDFORD AV,L,BMT,02/06/2016,19:00:00,REGULAR,2015738,798665,R23500-00-00


In [151]:
turn_list = np.asarray(bedford1.TURNSTILE.unique())
date_list = np.asarray(bedford.DATE.unique())

In [153]:
my_list = []
for d in date_list:
    for i in turn_list:
        t_max = max(bedford.EXITS[(bedford['TURNSTILE']==i) & (bedford['DATE']==d)])
        t_min = min(bedford.EXITS[(bedford['TURNSTILE']==i) & (bedford['DATE']==d)])
        out = t_max - t_min
        my_list.append(out)

In [154]:
turn_array = np.asarray(my_list).reshape(7,13)
turn_list = list(bedford.TURNSTILE.unique())

In [157]:
final = pd.DataFrame(turn_array,index = date_list, columns = turn_list).T
final

,02/06/2016,02/07/2016,02/08/2016,02/09/2016,02/10/2016,02/11/2016,02/12/2016
R23500-00-00,1042,598,870,986,1058,1000,1134
R23500-00-01,2226,1562,2094,2317,2319,2322,2397
R23500-03-00,3740,2594,2839,3072,3238,3190,3466
R23500-03-01,3122,2106,2472,2647,2757,2791,2976
R23500-03-02,2739,2066,2658,2900,3128,3125,3101
R23500-03-03,2454,1928,2833,3266,3103,3104,3016
R23500-03-04,792,677,961,988,1073,951,1026
R23500-06-00,16,12,17,19,34,19,26
R23500-06-01,77,49,60,6,169,87,81
R23500-06-02,3721,2462,2804,3715,3944,3596,3681
